In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import collections
import os
import random
import numpy as np
from tqdm import tqdm
import sys, email
import pandas as pd 
import math

In [2]:
os.getcwd()
os.chdir('C:/Users/Hoc/OneDrive/My Documents/Machine Learning/kaggle/ames/')
#os.chdir('C:/Users/Richard/OneDrive/My Documents/Machine Learning/kaggle/ames/')

In [16]:
train = pd.read_csv('train.csv', na_values='NA')
#print(train.head())

In [3]:
#########################################################
# Load Ames property cat descriptions dataset
#########################################################


# load enron dataset
import pandas as pd 
df = pd.read_csv('des.csv',encoding="utf8")
print(df.shape)
df.head()

(1460, 2)


Id                                                Des
0   1  . . Residential Low Density zoning classificat...
1   2  . . Residential Low Density zoning classificat...
2   3  . . Residential Low Density zoning classificat...
3   4  . . Residential Low Density zoning classificat...
4   5  . . Residential Low Density zoning classificat...

In [4]:
df = df.copy()

import string, re
# clean up subject line
df['Des'] = df['Des'].str.lower()
df['Des'] = df['Des'].str.replace(r'[^a-z]', ' ')  
df['Des'] = df['Des'].str.replace(r'\s+', ' ')  

In [5]:
# create sentence list 
df_text = df["Des"].tolist()# + ". " + df["Content"])

sentences = ' '.join(df_text)
words = sentences.split()

print('Data size', len(words))
 

# get unique words and map to glove set
print('Unique word count', len(set(words))) 
 

# drop rare words
vocabulary_size = 500

def build_dataset(words):
  count = [['unknown', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in tqdm(words):
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)

del words  
print('Most common words (+UNK)', count[:10])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Data size 284303
Unique word count 328


100%|█████████████████████████████████████████████████████████████████████| 284303/284303 [00:00<00:00, 2182916.77it/s]


Most common words (+UNK) [['unknown', 0], ('of', 11337), ('the', 10930), ('typical', 9246), ('quality', 8473), ('average', 7858), ('on', 6750), ('location', 6734), ('exterior', 6699), ('basement', 6577)]
Sample data [58, 84, 67, 59, 60, 1, 2, 11, 24, 112] ['residential', 'low', 'density', 'zoning', 'classification', 'of', 'the', 'sale', 'paved', 'alley']


In [6]:
####################################################################
# find matches with glove 
####################################################################
GLOVE_DATASET_PATH = 'C:/Users/Hoc/Downloads/glove.840B.300d/glove.840B.300d.txt'

from tqdm import tqdm
import string
embeddings_index = {}
f = open(GLOVE_DATASET_PATH, encoding="utf8") #need to specify encoding or error.
word_counter = 0
for line in tqdm(f):
  values = line.split()
  word = values[0]
  if word in dictionary:
    #print(values[1:])
    if '.com' in values[1]:
        continue
    else:
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
  word_counter += 1
f.close()

print('Found %s word vectors matching text data set.' % len(embeddings_index))
print('Total words in GloVe data set: %s' % word_counter)

2196017it [00:56, 39089.20it/s]


Found 325 word vectors matching text data set.
Total words in GloVe data set: 2196013


In [7]:
#########################################################
# Check out some clusters
#########################################################

# create a dataframe using the embedded vectors and attach the key word as row header
import pandas as pd
dataframe = pd.DataFrame(embeddings_index)
dataframe = pd.DataFrame.transpose(dataframe)
 
# See what it learns and look at clusters to pull out major themes in the data
CLUSTER_SIZE = 5
# cluster vector and investigate top groups
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=CLUSTER_SIZE)
cluster_make = kmeans.fit_predict(dataframe)

labels = kmeans.predict(dataframe)
import collections
cluster_frequency = collections.Counter(labels)
print(cluster_frequency)
cluster_frequency.most_common()

clusters = {}
n = 0
for item in labels:
    if item in clusters:
        clusters[item].append(list(dataframe.index)[n])
    else:
        clusters[item] = [list(dataframe.index)[n]]
    n += 1

#Ideally we would vary cluster size, inpsect and assign meaningfull group names but do on fly instead:
groups = []
#print the top cluster_size most common clusters
for k,v in cluster_frequency.most_common(100):
  print('\n\n')
  print('Cluster:', k)
  print (' '.join(clusters[k]))
  groups.append(clusters[k])

Counter({3: 155, 4: 57, 1: 56, 2: 30, 0: 27})



Cluster: 3
the and to of a in on with from was or not all has more one if just when some other new no than only over good two off home last down part between available site system s lot end family old water within car state hard quality side public story de members above below access face single short sale property hot major air half present living main interest clear inside section box material board terms purchase excellent feature court college yes estate quick unit regular trade gas sold allowed poor cash finish finished contract e described block warm score fair w completed payment properties mostly separate none etc rating forced sides roll university port officer attached minor mixed g privacy linked unknown split foundation covering warranty electricity rough tennis functionality damaged exceptional dot steam foreclosure amp floating quarters utilities allocation poured cracking deed deeds settling shakes tar zoning feeder deduct

In [8]:
print(labels)

[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 1 3 3 3 3 3 2 3 3 3 3 3 3 3 4 3 4 3 3 1 3 1 3 3 3 2 3 3 3 1 3 3 3 3 4 3 3
 3 3 3 3 3 3 1 3 3 3 2 2 3 2 1 3 3 3 1 2 4 1 3 3 3 4 3 3 3 4 3 3 1 3 3 3 1
 3 4 3 3 1 1 3 1 1 3 2 4 4 4 3 2 3 3 3 3 3 3 4 3 3 3 2 3 1 3 4 3 2 1 2 2 1
 3 1 3 1 1 3 1 3 3 2 3 1 3 3 2 3 3 1 1 4 1 3 2 3 4 3 4 3 3 1 1 1 3 1 1 3 4
 1 3 3 4 4 1 3 1 4 3 3 1 3 3 3 3 1 3 2 4 2 1 4 1 3 1 1 4 4 2 4 3 1 4 3 4 3
 4 4 4 4 4 3 3 4 1 4 1 4 1 0 1 1 4 1 2 1 3 3 4 2 4 2 4 2 1 3 3 3 3 4 4 2 2
 4 1 3 1 1 2 0 2 4 4 3 3 3 1 4 2 4 3 4 3 3 0 0 4 1 2 4 2 0 0 4 4 4 3 0 2 4
 0 4 0 1 4 3 0 0 0 4 0 1 4 0 0 1 0 0 0 0 0 0 4 0 0 0 0 0 0]


In [11]:
# boost bags with cosine distance from full glove data set
from tqdm import tqdm
import string
embeddings_index = {}
f = open(GLOVE_DATASET_PATH, encoding="utf8")
word_counter = 0
for line in tqdm(f):
    values = line.split()
    word = values[0]
  # difference here as we don't intersect words, we take most of them
    if '.com' in values[1]:
        continue
    elif (word.islower() and word.isalpha()): # work with smaller list of vectors
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    word_counter += 1
f.close()

print('Found %s word vectors matching text data set.' % len(embeddings_index))
print('Total words in GloVe data set: %s' % word_counter)

# create a dataframe using the embedded vectors and attach the key word as row header
glove_dataframe = pd.DataFrame(embeddings_index)
glove_dataframe = pd.DataFrame.transpose(glove_dataframe)

temp_matrix = pd.DataFrame.as_matrix(glove_dataframe)
import scipy
import scipy.spatial

vocab_boost_count = 5
for group_id in range(len(groups)):
  print('Working bag number:', str(group_id))
  glove_dataframe_temp = glove_dataframe.copy()
  vocab = []
  for word in groups[group_id]:
    print(word)
    vocab.append(word)
    cos_dist_rez = scipy.spatial.distance.cdist(temp_matrix, np.array(glove_dataframe.loc[word])[np.newaxis,:], metric='cosine')
    # find closest words to help
    glove_dataframe_temp['cdist'] = cos_dist_rez
    glove_dataframe_temp = glove_dataframe_temp.sort_values(['cdist'], ascending=[1])
    vocab = vocab + list(glove_dataframe_temp.head(vocab_boost_count).index)
  # replace boosted set to old department group and remove duplicates
  groups[group_id] = list(set(vocab))

# save final objects to disk
import pickle as pickle
with open('full_bags.pk', 'wb') as handle:
  pickle.dump(groups, handle)

2196017it [01:27, 25015.36it/s]


Found 419134 word vectors matching text data set.
Total words in GloVe data set: 2196001


C:\Users\Hoc\AppData\Local\conda\conda\envs\xgbDSv1\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Working bag number: 0
restorative
phagemid
cash
wеrе
juerga
pasific
reedobandito
lasat
equiprobable
tabouli
secluded
gbaji
frosts
oxcart
split
chemi
lounging
disinfectors
capcoms
semiskilled
zoning
kingz
husked
bertuah
amp
telling
bidentata
millbank
plethora
cher
functionality
gamification
hotrufftrade
frontin
violín
cracking
members
has
access
sublevel
pillowy
testacea
cms
fays
akashic
jacinta
inpulse
independiente
other
conciencia
gorse
dionysusapplication
fantagraphics
lit
nuyen
conveying
antiestrogen
overlimit
orgainzation
rescue
wvu
donnee
neuromasts
diagrs
thalaivar
ryerson
monteverde
theron
simultaneously
liposafe
openmsx
aparecem
trees
éventuellement
führen
palen
poxleitner
several
antidiabetes
bellwood
oversteering
rofling
breakers
faroese
fados
lamarcus
introtext
greal
spenny
disguises
hard
nickles
olimpo
denatures
evolves
cremes
sepration
dieseases
attainable
aquaintences
fennell
absolue
thhe
thework
benissa
monophysite
begrudges
nitinol
mimbar
tits
privacy
nationalised
clav

shingle
selichot
homes
suggested
tisa
wiring
veneer
viridula
asphalt
committe
farceur
amontillado
precipitously
humax
tradekey
constructed
highschoolers
pool
masonry
fulmer
omerta
sybols
building
stdc
villa
pronouncable
marchant
dominating
rescription
solides
vapour
reticuloendothelial
gige
veils
sized
nomadism
garage
extending
berkat
ytou
theunlovedbarbie
dabbing
unfinished
gifpeanutbutter
traff
succesfuly
decolletage
oxxo
driveway
ardell
withyou
appers
kaktus
condo
downstairs
jacked
furnace
interior
bionics
muhammed
oncology
meeting
exhaustiveness
sule
straplocks
convenince
seter
stranger
chronoscope
floor
impossilbe
interwork
thermoelement
plently
aigoo
mansions
reife
rooms
metalli
shed
billrubin
azabicyclo
toeach
filers
inspirationanaelle
cement
jodido
subscriber
nastier
colourblock
defition
gable
bedding
birdcalls
clay
blue
soudn
stucco
chevron
studen
gating
stone
fusili
crazed
cinder
mansard
exterior
ended
transformed
spitted
turboshaft
männer
longhaired
snortdrugs
ponying
roxxi


teleworking
mortgahe
banged
epergne
mangroves
eglington
sf
slotboss
touraments
molko
wimmens
reich
bovver
unfinshed
bagi
claussen
createthislookforless
gebeurd
nd
comtempt
gimmik
lumines
bloomington
draw
concerete
affirmations
synchronizer
ansi
ethicist
marcona
theimaginationfoundation
brookside
polskie
aptosid
libya


In [17]:
##################################################################### 
# Create features of word counts for each item in each text
#####################################################################
 
import pickle as pickle
with open('full_bags.pk', 'rb') as handle:
    groups = pickle.load(handle)

# loop through all texts and count group words in each raw text
words_groups = []
for group_id in range(len(groups)):
  work_group = []
  print('Working bag number:', str(group_id))
  top_words = groups[group_id]
  for index, row in tqdm(df.iterrows()): 
    text = (row["Des"]) 
    work_group.append(len(set(top_words) & set(text.split())))
    #work_group.append(len([w for w in text.split() if w in set(top_words)]))
    
  words_groups.append(work_group)

# count emails per category group and feature engineering

raw_text = []
text_length = []
text_word_count = []
group_1 = []
group_2 = []
group_3 = []
group_4 = []
group_5 = []
#group_6 = []
#group_7 = []
#group_8 = []
#group_9 = []
#group_10 = []
#group_11 = []
#group_12 = []
#group_13 = []
#group_14 = []
#group_15 = []
#group_16 = []
#group_17 = []
#group_18 = []
#group_19 = []
#group_20 = []
#group_21 = []
#group_22 = []
#group_23 = []
#group_24 = []
#group_25 = []
#group_26 = []
#group_27 = []
#group_28 = []
#group_29 = []
#group_30 = []

final_outcome = []

df['Des'].fillna('', inplace=True)

counter = 0
for index, row in tqdm(df.iterrows()):
  raw_text.append([row["Des"]])
  group_1.append(words_groups[0][counter])
  group_2.append(words_groups[1][counter]) 
  group_3.append(words_groups[2][counter])
  group_4.append(words_groups[3][counter])
  group_5.append(words_groups[4][counter])
  outcome_tots = [words_groups[0][counter], words_groups[1][counter], words_groups[2][counter],
    words_groups[3][counter], words_groups[4][counter]] 
  final_outcome.append(outcome_tots.index(max(outcome_tots)))
    
  text_length.append(len(row['Des']))
  text_word_count.append(len(row['Des'].split()))
 
  counter += 1


# add simple engineered features?
training_set = pd.DataFrame({
              "raw_text":raw_text,
              "group_1":group_1,
              "group_2":group_2,
              "group_3":group_3,
              "group_4":group_4,
              "group_5":group_5,
              "text_length":text_length,
              "text_word_count":text_word_count,
              "outcome":train.SalePrice})

# might want to remove all texts that have all zeros (i.e. not from any of required categories)
#training_set = training_set[
#              (training_set.group_1 > 0) | 
#              (training_set.group_2 > 0) |
#              (training_set.group_3 > 0) |
#              (training_set.group_4 > 0) |
#              (training_set.group_5 > 0)]
#print(len(training_set))

# save extractions to file
training_set.to_csv('text_classification_df.csv', index=False, header=True)

Working bag number: 0


1460it [00:00, 3316.29it/s]


Working bag number: 1


1460it [00:00, 4876.98it/s]


Working bag number: 2


1460it [00:00, 4867.15it/s]


Working bag number: 3


1460it [00:00, 6840.44it/s]


Working bag number: 4


1460it [00:00, 6956.94it/s]
1460it [00:00, 10438.23it/s]


In [30]:
####################################################
# TensorFlow Deep Classifier 
####################################################
# https://www.tensorflow.org/api_docs/python/tf/contrib/learn/DNNClassifier
# create a wide and deep model and also predict a few entries

#Add Label column + extra columns to dataset prior to read in.
model_ready_data = pd.read_csv('text_classification_df.csv') 

# https://stackoverflow.com/questions/38250710/how-to-split-data-into-3-sets-train-validation-and-test
# (60% - train set, 20% - validation set, 20% - test set)
df_train, df_test, df_val = np.split(model_ready_data.sample(frac=1), [int(.6*len(model_ready_data)), int(.8*len(model_ready_data))])

# Continuous base columns
text_length = tf.contrib.layers.real_valued_column("text_length")
text_word_count = tf.contrib.layers.real_valued_column("text_word_count")

group_1 = tf.contrib.layers.real_valued_column("group_1")
group_2 = tf.contrib.layers.real_valued_column("group_2")
group_3 = tf.contrib.layers.real_valued_column("group_3")
group_4 = tf.contrib.layers.real_valued_column("group_4")
group_5 = tf.contrib.layers.real_valued_column("group_5")

text_length_bucket = tf.contrib.layers.bucketized_column(text_length, boundaries=[900, 1200, 1500, 1800, 1900])

# Categorical base columns
#is_AM_sparse_column = tf.contrib.layers.sparse_column_with_keys(column_name="is_AM", keys=["yes", "no"])
# is_AM = tf.contrib.layers.one_hot_column(is_AM_sparse_column)\
#is_weekday_sparse_column = tf.contrib.layers.sparse_column_with_keys(column_name="is_weekday", keys=["yes", "no"])
# is_weekday = tf.contrib.layers.one_hot_column(is_weekday_sparse_column)

categorical_columns = [text_length_bucket] 

deep_columns = [text_length, text_word_count,
               group_1, group_2, group_3, group_4, 
               group_5]

simple_columns = [group_1, group_2, group_3, group_4, 
               group_5]

import tempfile
model_dir = tempfile.mkdtemp()
regressor = tf.contrib.learn.DNNRegressor(feature_columns=simple_columns,
                                hidden_units=[20, 20],
                                #n_classes=6, # think only for classifiers
                                optimizer=tf.train.ProximalAdagradOptimizer,
                                #learning_rate=0.1,
                                #l1_regularization_strength=0.001,
                                model_dir=model_dir)

# Define the column names for the data sets.
COLUMNS = ['text_length',
 'text_word_count',
 'group_1',
 'group_2',
 'group_3',
 'group_4',
 'group_5',
 'outcome']
LABEL_COLUMN = 'outcome'
#CATEGORICAL_COLUMNS = ["is_AM", "is_weekday"]
CONTINUOUS_COLUMNS = ['text_length',
 'text_word_count',
 'group_1',
 'group_2',
 'group_3',
 'group_4',
 'group_5']

#LABELS = [0, 1, 2, 3, 4, 5]
 
def input_fn(df):
  # Creates a dictionary mapping from each continuous feature column name (k) to
  # the values of that column stored in a constant Tensor.
  continuous_cols = {k: tf.constant(df[k].values)
                     for k in CONTINUOUS_COLUMNS}
  # Creates a dictionary mapping from each categorical feature column name (k)
  # to the values of that column stored in a tf.SparseTensor.
  categorical_cols = {k: tf.SparseTensor(
      indices=[[i, 0] for i in range(df[k].size)],
      values=df[k].values,
      dense_shape=[df[k].size, 1])
                      for k in CATEGORICAL_COLUMNS}
  # Merges the two dictionaries into one.
  feature_cols = dict(continuous_cols.items() + categorical_cols.items())
  
  # Converts the label column into a constant Tensor.
  label = tf.constant(df[LABEL_COLUMN].values)
  # Returns the feature columns and the label.
  return feature_cols, label

def train_input_fn():
  return input_fn(df_train)

def eval_input_fn():
  return input_fn(df_test)
# After reading in the data, you can train and evaluate the model:

regressor.fit(input_fn=train_input_fn, steps=200)
results = regressor.evaluate(input_fn=eval_input_fn, steps=1)
for key in sorted(results):
    print("%s: %s" % (key, results[key]))

y_pred = regressor.predict(input_fn=lambda: input_fn(df_val), as_iterable=False)
print(y_pred)

print('buckets found:',set(y_pred))

# # confusion matrix analysis
# from sklearn.metrics import confusion_matrix
# confusion_matrix(df_val[LABEL_COLUMN], y_pred)
# pd.crosstab(df_val[LABEL_COLUMN], y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

# create some data
# https://stats.stackexchange.com/questions/95209/how-can-i-interpret-sklearn-confusion-matrix

#lookup = {0: 'group_1', 1:'group_2', 2:'group_3', 3:'group_4', 4:'group_5'}
#y_truet = pd.Series([lookup[_] for _ in df_val[LABEL_COLUMN]])
#y_predt = pd.Series([lookup[_] for _ in y_pred])
#pd.crosstab(y_truet, y_predt, rownames=['Actual'], colnames=['Predicted'], margins=True)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002DFBDB2E668>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\Hoc\\AppData\\Local\\Temp\\tmpdau9xwdr'}


TypeError: unsupported operand type(s) for +: 'dict_items' and 'dict_items'

In [33]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [34]:
model_ready_data = pd.read_csv('text_classification_df.csv')
model_ready_data.columns

Index(['raw_text', 'group_1', 'group_2', 'group_3', 'group_4', 'group_5',
       'text_length', 'text_word_count', 'outcome'],
      dtype='object')

In [50]:
X = model_ready_data.iloc[:,1:8] #NB. less final column! hence 1:8 not 1:7. (and 1 becuase we leave out raw text)
Y = model_ready_data.iloc[:,8]

In [51]:
X.columns

Index(['group_1', 'group_2', 'group_3', 'group_4', 'group_5', 'text_length',
       'text_word_count'],
      dtype='object')

In [52]:
Y = np.log1p(Y)

In [53]:
Y.head()

0    12.247699
1    12.109016
2    12.317171
3    11.849405
4    12.429220
Name: outcome, dtype: float64

In [57]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(7, input_dim=7, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [63]:
# fix random seed for reproducibility
# increased epochs from 100 to 1000
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=1000, batch_size=5, verbose=0)

In [66]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X.values, Y.values, cv=kfold, n_jobs=4)
print(results)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

[-0.75684767 -0.6334353  -0.52691228 -1.08976217 -0.80109809 -0.64104566
 -0.67699946 -0.937082   -0.79695826 -0.85329479]
Results: -0.77 (0.16) MSE
